In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
from agent.scalable_model import ScalableModel
from agent.data_downloader import get_data
from tics_config import tics_176
from agent.helper import plot

# Dowload Data

In [ ]:
tics = tics_176
train_start_date= '2009-01-01'
train_end_date= '2020-07-01'
test_start_date= '2020-07-01'
test_end_date= '2024-10-01'
config = ... #TODO

In [ ]:
data = get_data(tics, train_start_date, test_end_date)

# Create model

In [ ]:
model = ScalableModel(config)
model.load_data(data)

# split tics into groups

In [ ]:
tics_grouped = model.split(tics, train_start_date, train_end_date)

# Train sub models

In [ ]:
for tics_group in tics_grouped:
    model.train_sub(tics_group, train_start_date, train_end_date)


# (optional) Test sub models

In [ ]:
sub_test_result = model.test_sub(tics_grouped[0], test_start_date, test_end_date)

# Train the manger

In [ ]:
model.train(tics, train_start_date, train_end_date)

# Test the model (back test)

In [ ]:
manager_result = model.test(test_start_date, test_end_date, backtest=False, weights=False, valuse=True)
manger_values = manager_result['account_values']

manager_result['backtest']

# Test the model (Compart acount values)

In [ ]:
sub_values = []

for tics_group in tics_grouped:
    sub_values.append(model.test_sub(tics_group, test_start_date, test_end_date, backtest=False, weights=False, valuse=True))

df_baseline = pd.read_csv(Path("acount_value") / "dow_jones_data.csv")[['close', 'date']].rename(columns={'close': 'account_value'})
fig1, fig2 = plot(df_baseline, sub_values, manager_result, test_start_date, test_end_date)

In [ ]:
fig1.show()

In [ ]:
fig2.show()